In [1]:
import sys
sys.path

['c:\\big16\\trading_project\\ssuyan\\eBEST_real_data',
 'c:\\Users\\green\\anaconda3\\envs\\trading_32\\python37.zip',
 'c:\\Users\\green\\anaconda3\\envs\\trading_32\\DLLs',
 'c:\\Users\\green\\anaconda3\\envs\\trading_32\\lib',
 'c:\\Users\\green\\anaconda3\\envs\\trading_32',
 '',
 'c:\\Users\\green\\anaconda3\\envs\\trading_32\\lib\\site-packages',
 'c:\\Users\\green\\anaconda3\\envs\\trading_32\\lib\\site-packages\\win32',
 'c:\\Users\\green\\anaconda3\\envs\\trading_32\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\green\\anaconda3\\envs\\trading_32\\lib\\site-packages\\Pythonwin',
 'c:\\Users\\green\\anaconda3\\envs\\trading_32\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\green\\.ipython']

In [2]:
import threading
import queue
import pythoncom
 
from name_yalsooni.crawler_py.ebest.util import Log
from abc import *
 
 
# 병행처리기 - 추상화
class ThreadJob(threading.Thread):
    __metaclass__ = ABCMeta
 
    CM_COMMAND = "COMMAND"
    CM_SHUTDOWN = "SHUTDOWN"
 
    # 초기화
    @abstractmethod
    def _init(self):
        pass
 
    # 실행
    @abstractmethod
    def _execute(self, command):
        pass
 
    # 셧다운
    @abstractmethod
    def _shutdown(self):
        pass
 
    # 생성자
    def __init__(self, thread_name, command_queue_timeout):
        threading.Thread.__init__(self, name=thread_name)
        self.running = True
        self.commandQueue = queue.Queue()
        self.command_queue_timeout = command_queue_timeout
 
    # Thread start..
    def run(self):
        pythoncom.CoInitialize()
        self._init()
        self._waiting_queue()
 
    # 셧다운 명령어 요청
    def shutdown_call(self):
        self._push_command(self.CM_SHUTDOWN)
 
    # 명령어 대기
    def _waiting_queue(self):
        while self.running:
            try:
                command = self.commandQueue.get(True, self.command_queue_timeout)
            except queue.Empty as em:
                pythoncom.PumpWaitingMessages()
                continue
 
            Log.write("Request Command : " + command[self.CM_COMMAND])
            if command[self.CM_COMMAND] == self.CM_SHUTDOWN:
                self._shutdown()
                pythoncom.CoUninitialize()
            else:
                try:
                    self._execute(command)
                except Exception as ex:
                    Log.write("*EXCEPTION* EXECUTE COMMAND : " + str(ex))
 
    # 명령어를 큐에 삽입
    def _push_command(self, command):
        self.commandQueue.put(command)

In [3]:
import pythoncom
import win32com.client as win_client
import win32com.client as winAPI
 
from name_yalsooni.crawler_py.ebest.util import Log
 
import datetime
import time

STAND_BY = 0
RECEIVED = 1


class XASessionEvents:
    login_state = STAND_BY

    def OnLogin(self, code, msg):
        XASessionEvents.login_state = RECEIVED
        print(msg)

    def OnDisconnect(self, code, msg):
        pass

class XAQueryEvents:
    query_state = STAND_BY

    def OnReceiveData(self, code):
        XAQueryEvents.query_state = RECEIVED

    def OnReceiveMessage(self, error, nMessageCode, szMessage):
        print(szMessage)


SERVER_PORT = 20001
SHOW_CERTIFICATE_ERROR_DIALOG = False
REPEATED_DATA_QUERY = 1
TRANSACTION_REQUEST_EXCESS = -21
TODAY = datetime.datetime.now().strftime('%Y%m%d') 
 
class XASessionEventHandler:
    login_state = 0
    connect_state = 0
    login_flag = False
 
    def OnLogin(self, code, msg):
        if code == "0000":
            Log.write("login success")
            Log.write(str(msg))
            XASessionEventHandler.login_flag = True
            XASessionEventHandler.login_state = 1
            XASessionEventHandler.connect_state = 1
        else:
            Log.write("login fail")
            Log.write(str(msg))
            XASessionEventHandler.login_state = 2
 
    def OnDisconnect(self):
        XASessionEventHandler.login_flag = False
        XASessionEventHandler.login_state = 0
        XASessionEventHandler.connect_state = 0
        Log.write("Disconnect Server")
 
 
if __name__ == "__main__":
    id = "ssuyan"
    password = "tndus1!!"
    certificate_password = "qkrtndus1!!"
    xa_session = winAPI.DispatchWithEvents("XA_Session.XASession", XASessionEvents)

    # demo.ebestsec.co.kr => 모의투자 
    # hts.ebestsec.co.kr => 실투자
    xa_session.ConnectServer("hts.ebestsec.co.kr", SERVER_PORT)
    xa_session.Login(id, password, certificate_password, SERVER_PORT, SHOW_CERTIFICATE_ERROR_DIALOG)

    while XASessionEvents.login_state is STAND_BY:
        pythoncom.PumpWaitingMessages()
    XASessionEvents.login_state = STAND_BY

                                                      S2008 폐지된 인증서입니다.


In [4]:
import threading
import time
 
from name_yalsooni.crawler_py.ebest.xasession import XAConnector
from name_yalsooni.crawler_py.ebest.xasession import XASessionEventHandler
from name_yalsooni.crawler_py.ebest.util import Log
from name_yalsooni.crawler_py.ebest.definition import ThreadJob
 
 
# 연결 관리
class ConnectionManager(ThreadJob):
 
    _CM_LOGIN = "LOGIN"
    _CM_LOGOUT = "LOGOUT"
 
    _xa_connector = None
    _login_result = None
 
    def __init__(self):
        ThreadJob.__init__(self, "ConnectionManager", 10)
        self.lock = threading.Lock()
 
    # 스레드 초기화
    def _init(self):
        Log.write("ConnectionManager init..")
        self._xa_connector = XAConnector()
 
    # 스레드 수행
    def _execute(self, command):
        with self.lock:
            if command[self.CM_COMMAND] == self._CM_LOGIN:
                self._xa_connect()
            elif command[self.CM_COMMAND] == self._CM_LOGOUT:
                self._xa_disconnect()

 
    # 스레드 종료
    def _shutdown(self):
        pass
 
    # 연결 하기
    def _xa_connect(self):
        if self._xa_connector.is_connected() is False:
            self._xa_connector.connect_server()
            self._login_result = self._xa_connector.login()
            # if self.__login_result:
                # account_list = self.__xa_connector.get_account_list()
                # for account in account_list:
                #     Log.write("account number : " + account)
 
        return self._login_result
 
    # 연결 해제
    def _xa_disconnect(self):
        self._xa_connector.disconnect_server()
 
    # 연결 응답 대기
    def _waiting_login(self):
        while XASessionEventHandler.login_state == 0:
            time.sleep(0.2)
 
    # 연결 해제 응답 대기
    def _waiting_logout(self):
        while XASessionEventHandler.login_state != 0:
            time.sleep(0.2)
 
    # 연결 요청
    def login_call(self):
        command = dict()
        command[self.CM_COMMAND] = self._CM_LOGIN
        self._push_command(command)
        self._waiting_login()
 
    # 연결 해제 요청
    def disconnect_call(self):
        command = dict()
        command[self.CM_COMMAND] = self._CM_LOGOUT
        self._push_command(command)
        self._waiting_logout()
 
 
class ConnectionManagerFactory:
 
    _connection_manager = ConnectionManager()
 
    @staticmethod
    def get_instance():
        return ConnectionManagerFactory._connection_manager

In [5]:
from abc import *
 
from name_yalsooni.crawler_py.ebest.definition import ThreadJob
from name_yalsooni.crawler_py.ebest.connection import ConnectionManagerFactory
from name_yalsooni.crawler_py.ebest.util import Log
from name_yalsooni.crawler_py.ebest.xasession import XASessionEventHandler
 
 
class EventHandler:
    event = None
 
 
class QueryThreadJob(ThreadJob):
    __metaclass__ = ABCMeta
 
    _OP_CALL = "CALL"
 
    _event = None
    _thread_name = None
 
    _operation_dict = None
    _connection_manager = None
 
    @abstractmethod
    def _operation_init(self):
        pass
 
    @abstractmethod
    def _operation_call(self):
        pass
 
    def __init__(self, thread_name, command_queue_timeout):
        ThreadJob.__init__(self, thread_name, command_queue_timeout)
        self._thread_name = thread_name
        self._operation_dict = dict()
 
    def _init(self):
        Log.write(self._thread_name + " init..")
        self._connection_manager = ConnectionManagerFactory.get_instance()
 
        #기본 오퍼레이션 등록
        self._operation_dict[self._OP_CALL] = self._operation_call
        self._operation_init()
 
    def _execute(self, command):
        if not XASessionEventHandler.login_flag:
            self._connection_manager.login_call()
 
        try:
            operation = self._operation_dict[command[self.CM_COMMAND]]
            operation(command)
        except KeyError:
            Log.write("Not found operation key : " + command[self.CM_COMMAND])
 
    def _shutdown(self):
        self._event.UnadviseRealData()
 
    # push call command
    def call(self, command):
        command[self.CM_COMMAND] = self._OP_CALL
        self._push_command(command)

In [6]:
from name_yalsooni.crawler_py.ebest.queryjob import EventHandler
from name_yalsooni.crawler_py.ebest.queryjob import QueryThreadJob
from name_yalsooni.crawler_py.ebest.util import Log, EventUtil
 
 # 해당 증권사의 tr은 종목코드를 요청값으로 보내줘야한다

class S3EventHandler(EventHandler):
    def OnReceiveRealData(self, code):
        shcode = self.event.GetFieldData("OutBlock", "shcode")
        price = self.event.GetFieldData("OutBlock", "price")
        Log.write(code + " : " + shcode + " - " + price)
 
 
class S3(QueryThreadJob):
    _OP_APPEND = "APPEND"
    CM_SHCODE = "SHCODE"
 
    def __init__(self):
        QueryThreadJob.__init__(self, "RT_S3_KOSPI", 0.1)
 
    def _operation_init(self):
        self._operation_dict[self._OP_APPEND] = self._operation_append
 
    def _operation_call(self, command):
        self._event = EventUtil.get_event("S3_", S3EventHandler)
        self._event.SetFieldData("InBlock", "shcode", command[self.CM_SHCODE])
        self._event.AdviseRealData()
        Log.write("Request done : " + command[self.CM_SHCODE])
 
    def _operation_append(self, command):
        self._event.SetFieldData("InBlock", "shcode", command[self.CM_SHCODE])
        self._event.AdviseRealData()
        Log.write("Request done : " + command[self.CM_SHCODE])
 
    # push append command
    def append_call(self, command):
        command[self.CM_COMMAND] = self._OP_APPEND
        self._push_command(command)

In [7]:
import time
from time import sleep
from threading import Thread
 
from name_yalsooni.crawler_py.ebest.connection import ConnectionManagerFactory
from name_yalsooni.crawler_py.ebest.query.nws import NWS
from name_yalsooni.crawler_py.ebest.query.s3 import S3
from name_yalsooni.crawler_py.ebest.util import Log
 
 
class RunnerRealTime:
 
    cm = None
 
    def execute(self):
        Log.write("Process Start Up..")
        Log.write("-- ConnectionManager Start --")
        self.cm = ConnectionManagerFactory.get_instance()
        self.cm.start()
 
        Log.write("-- [NWS] NEWS Start --")
        nws_event = NWS()
        nws_event.start()
 
        Log.write("-- [S3_] KOSPI Start --")
        s3_event = S3()
        s3_event.start()
 
        Log.write("-- Push Command --")
        # 뉴스 요청
        nws_command = dict()
        nws_command[nws_event.CM_NWCODE] = "NWS001"
        nws_event.call(nws_command)
 
        # 한화에어로 체결정보 요청
        s3_command = dict()
        s3_command[s3_event.CM_SHCODE] = "012450"
        s3_event.call(s3_command)
 
        # 10초 지연 후 APPEND 명령어
        time.sleep(10)

        # 한화에어로 체결정보 요청
        s3_command[s3_event.CM_SHCODE] = "012450"
        s3_event.append_call(s3_command)
 
        while True:
            time.sleep(5)
             
 
def main():
    RunnerRealTime().execute()
 
 
if __name__ == "__main__":
    main()

[2023-07-15 15:19:44][MainThread] Process Start Up..
[2023-07-15 15:19:44][MainThread] -- ConnectionManager Start --
[2023-07-15 15:19:44][MainThread] -- [NWS] NEWS Start --
[2023-07-15 15:19:44][ConnectionManager] ConnectionManager init..
[2023-07-15 15:19:44][MainThread] -- [S3_] KOSPI Start --
[2023-07-15 15:19:44][RT_NWS] RT_NWS init..
[2023-07-15 15:19:45][RT_S3_KOSPI] RT_S3_KOSPI init..
[2023-07-15 15:19:45][MainThread] -- Push Command --
[2023-07-15 15:19:45][RT_NWS] Request Command : CALL
[2023-07-15 15:19:45][ConnectionManager] Request Command : LOGIN
[2023-07-15 15:19:45][RT_S3_KOSPI] Request Command : CALL


AttributeError: 'S3' object has no attribute 'append_call'